In [3]:
import pandas as pd 
dataset = pd.read_csv("data.csv") 
print(dataset.head())
en_sentences = dataset["EN"].tolist()
fr_sentences = dataset["FR"].tolist()

                                   EN                                  FR
0           i m at a loss for words .               j en perds mes mots .
1           i m at a loss for words .              les mots me manquent .
2           i m at a loss for words .         je ne trouve pas les mots .
3  you re in better shape than i am .  tu es en meilleure forme que moi .
4                 you are in my way .              tu es sur mon chemin .


In [ ]:
from collections import Counter 

class Vocab:
    def __init__(self):
        self.max_len = 30
        self.token_to_ids = {"<UNK>": 0}
        self.id_to_token = {0: "<UNK>"}
        self.freq = Counter()
        self.special_tokens = ["<pad>", "<sos>", "<eos>"]
        for token in self.special_tokens:
            self.add_token(token)

    def add_token(self, token):
        if token not in self.token_to_ids:
            id = len(self.token_to_ids)
            self.token_to_ids[token] = id
            self.idx_to_token[id] = token 
    
    def sentence_to_id(self, sentence, maxlen):
        id_list = []
        words = ["<sos>"] + sentence.split() + ["<eos>"]

        for word in words:
            if word in self.token_to_ids.keys():
                id_list.append(self.token_to_ids[word])
            else:
                id_list.append(self.token_to_ids["<UNK>"])

        if len(id_list) > maxlen:
            id_list = id_list[:maxlen]
        while len(id_list) < maxlen:
            id_list.append(self.token_to_ids["<pad>"])
        
        return id_list


    def build_Vocab(self, sentences):
    
        count = Counter()
        for sentence in sentences:
            tokens = sentence.lower().split()
            for token in tokens:
                counter += 1
                self.add_token(token)